In [1]:
import pandas as pd
from scripts.mock.mock_cleaning import create_demand
import pyomo.environ as pyo

# Data import

In [2]:
nodes = pd.read_pickle("../../data/original/nodes.pkl")
channels = pd.read_pickle("../../data/original/channels.pkl")

In [3]:
#nodes.to_csv("../../data/original/nodes.csv")
#channels.to_csv("../../data/original/channels.csv")

## Modeling

Change the logic of CHANNELS set construction by considering the pair of nodes as the edge identifier.
In this way I also have to deal with:
- [X] multiedges, aka multiple edges between two nodes. I can not consider those for simplicity
- [X] directed relationship. I need to understand if flipping the channels - as done before - can be reasonable. I also need to determine if the flipped pair of nodes is considered a valid index by pyomo.

In [4]:
### Note that the following are arbitrary policies, that can be changed as needed.
### Deal with multi-edges (aka multiple channels between two peers):
### - average rate fee
### - average base fee,
### - average capacity
### - keep one of the two channel ids
#aggregation_dict = {
#    "channel_id": "first",
#    "rate_fee": "mean",
#    "base_fee": "mean",
#    "capacity": "sum"
#}
#channels.reset_index(inplace=True)
#channels = channels.groupby(["node1_pub", "node2_pub"]).agg(aggregation_dict)
#channels.reset_index(inplace=True)
#channels.set_index("channel_id", inplace=True)

In [5]:
### To delete from [outgoing] and [incoming] lists the multi-edge
#for i in nodes.index:
#    chan = []
#    for l in nodes.loc[i, "outgoing_channels"]:
#        if l in channels.index:
#            chan.append(l)
#    nodes.at[i, "outgoing_channels"] = chan
#
#for i in nodes.index:
#    chan = []
#    for l in nodes.loc[i, "incoming_channels"]:
#        if l in channels.index:
#            chan.append(l)
#    nodes.at[i, "incoming_channels"] = chan

In [6]:
## Add to the nodes dataset the list of peers (nodes who have a channel with the N node)
#channels.reset_index(inplace=True)
#channels.set_index(["node1_pub", "node2_pub"], inplace=True)
#nodes["peers"] = None
#for n in nodes.index:
#    peers = []
#    for c in nodes.loc[n, "outgoing_channels"]:
#        source, destination = channels[channels["channel_id"]==c].index[0]
#        #print(f"For node {n}, the channel {c} is {source} --> {destination}")
#        try:
#            assert source == n
#            peers.append(destination)
#        except AssertionError as e:
#            print(f"{source} is NOT equal to {n} for channel {c}")
#    #print(peers)
#    nodes.at[n, "peers"] = peers
#channels.reset_index(inplace=True)
#channels.set_index("channel_id", inplace=True)

In [7]:
model = pyo.ConcreteModel(name="Min cost flow problem")
model.NODES = pyo.Set(initialize=nodes.index)
model.CHANNELS = pyo.Set(initialize=[(channels.loc[i, "node1_pub"], channels.loc[i, "node2_pub"]) for i in channels.index])

In [8]:
nodes = create_demand(nodes, 100000)

Transaction of 100000 sats.
Sender: gocharlie-lightning-node-1
Receiver: the_holy_cheese_grater.


In [9]:
model.x = pyo.Var(model.CHANNELS, domain=pyo.Binary)
model.a = pyo.Var(model.CHANNELS, domain=pyo.NonNegativeReals, bounds=(0, max(nodes["demand"])))

In [10]:
channels.reset_index(inplace=True)
channels.set_index(["node1_pub", "node2_pub"], inplace=True)
channels.sort_index(inplace=True)

In [11]:
def objective_function(model: pyo.ConcreteModel):
    return sum(model.a[i] * channels.loc[i, "rate_fee"] for i in model.CHANNELS) + sum(model.x[i] * channels.loc[i, "base_fee"] for i in model.CHANNELS)

model.totalCost = pyo.Objective(rule=objective_function(model), sense=pyo.minimize)

### Constraints

#### Number of paths rule

This constraint enforces the symmetrical splitting into different channels of the amount along the path. Specifically, the number of used incoming channels for a node shall be equal to the number of used outgoing channels.

$$
\sum_{(i,n) \in E} x_{i,n} - \sum_{(n,j) \in E} x_{n,j} = 0 \text{ } \forall n \in V
$$


In [12]:
def number_path_rule(model: pyo.ConcreteModel, n):
    outgoing = [model.x[(i, j)] for i, j in channels.index if i == n]
    incoming = [model.x[(i, j)] for i, j in channels.index if j == n]
    return sum(incoming) == sum(outgoing)

model.NumberPathConstraint = pyo.Constraint(model.NODES, rule=number_path_rule, name="Number path constraint")

#### Capacity constraint

$$amount_{i,j} \le capacity_{i,j} \times x_{i,j} \text{ } \forall (i,j) \in E$$

In [13]:
def capacity_constraint(model: pyo.ConcreteModel, a, b):
    return model.a[(a, b)] <=  channels.loc[(a, b), "capacity"] * model.x[(a, b)]

model.CapacityConstraint = pyo.Constraint(model.CHANNELS, rule=capacity_constraint, name="Capacity constraint")

#### Flow balance constraint

$$\sum_{(s,i) \in E} amount_{si} - \sum_{(i,t) \in E} amount_{it} = b_i \text{ } \forall i \in V$$


In [14]:
channels.reset_index(inplace=True)
channels.set_index("channel_id", inplace=True)

def flow_balance_constraint(model: pyo.ConcreteModel, n: str):
    InFlow = sum(model.a[(channels.loc[a, "node1_pub"], channels.loc[a, "node2_pub"])] for a in nodes.loc[n, 'incoming_channels'])
    OutFlow = sum(model.a[(channels.loc[a, "node1_pub"], channels.loc[a, "node2_pub"])] for a in nodes.loc[n, 'outgoing_channels'])
    return  OutFlow + nodes.loc[n, "demand"] == InFlow

model.FlowBalanceConstraint = pyo.Constraint(model.NODES, rule=flow_balance_constraint, name="Flow balance constrain")

channels.reset_index(inplace=True)
channels.set_index(["node1_pub", "node2_pub"], inplace=True)
channels.sort_index(inplace=True) 

## Solving the model

In [15]:
opt = pyo.SolverFactory('cbc')
results = opt.solve(model, tee=True)

if (results.solver.status == pyo.SolverStatus.ok) and (results.solver.termination_condition == pyo.TerminationCondition.optimal):
    print('\nOptimal solution found')
elif results.solver.termination_condition == pyo.TerminationCondition.feasible:
    print('\nFeasible but not proven optimal solution found')
elif results.solver.termination_condition == pyo.TerminationCondition.infeasible:
    raise Exception("The model is infeasible")
else:
    print('\nSolver Status: ',  results.solver.status)
    raise Exception(results.solver.status)

print('\nObject function value = ', model.Objective())


Welcome to the CBC MILP Solver 
Version: 2.10.8 
Build Date: May  9 2022 

command line - /usr/bin/cbc -printingOptions all -import /tmp/tmp8jbulb2q.pyomo.lp -stat=1 -solve -solu /tmp/tmp8jbulb2q.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 51411 (-7319) rows, 89054 (-4942) columns and 257450 (-23778) elements
Statistics for presolved model
Original problem has 46998 integers (46998 of which binary)
Presolved problem has 44527 integers (44527 of which binary)
==== 20800 zero objective 3049 different
==== absolute objective values 3049 different
==== for integers 17284 zero objective 794 different
==== for integers absolute objective values 794 different
===== end objective counts


Problem has 51411 rows, 89054 columns (68254 with objective) and 257450 elements
There are 4502 singletons with objective 
Column breakdown:
0 of type 0.0->inf, 44527 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 o

In [30]:
from decimal import Decimal
DF_channels = pd.DataFrame()
c = 0
for index, value in model.a.extract_values().items():
    if value != 0:
        DF_channels.loc[c, "source"] = index[0]
        DF_channels.loc[c, "destination"] = index[1]
        DF_channels.loc[c, "source-alias"] = nodes.loc[index[0], "alias"]
        DF_channels.loc[c, "destination-alias"] = nodes.loc[index[1], "alias"]
        DF_channels.loc[c, "capacity"] = channels.loc[index, "capacity"]
        DF_channels.loc[c, "amount"] = Decimal(value)
        DF_channels.loc[c, "base_fee"] = channels.loc[(index[0], index[1]), "base_fee"]
        DF_channels.loc[c, "rate_fee"] = channels.loc[(index[0], index[1]), "rate_fee"]
        c += 1

DF_channels_pos = DF_channels[DF_channels["amount"]!=0]
DF_channels_pos

,source,destination,source-alias,destination-alias,capacity,amount,base_fee,rate_fee
0,02881003525adb05fef7f73eacc8f31903791ced708a4f...,035e4ff418fc8b5554c5d9eea66396c227bd429a3251c8...,gocharlie-lightning-node-1,WalletOfSatoshi.com,5000000.0,100000,0.0,0.000010
1,029267159e8ed64dc44f1deda34c918f45c7ba2d6b2533...,029efe15ef5f0fcc2fdd6b910405e78056b28c9b64e1fe...,Ludwig,adam.masterofpearls.net,5933222.0,100000,0.0,0.000000
2,029efe15ef5f0fcc2fdd6b910405e78056b28c9b64e1fe...,039174f846626c6053ba80f5443d0db33da384f1dde135...,adam.masterofpearls.net,e960fd8385a1603003727,2000000.0,100000,0.0,0.000000
3,032d5a4b5a6a344ca15f6284e3e149f4716a1af782ffbb...,0364913d18a19c671bb36dd04d6ad5be0fe8f2894314c3...,tka.jp,LQwD-Canada,10000000.0,100000,1.0,0.000001
4,035e4ff418fc8b5554c5d9eea66396c227bd429a3251c8...,029267159e8ed64dc44f1deda34c918f45c7ba2d6b2533...,WalletOfSatoshi.com,Ludwig,10000000.0,100000,0.0,0.000000
5,0364913d18a19c671bb36dd04d6ad5be0fe8f2894314c3...,02cca6c5c966fcf61d121e3a70e03a1cd9eeeea024b26e...,LQwD-Canada,the_holy_cheese_grater,2000000.0,100000,1.0,0.000005
6,039174f846626c6053ba80f5443d0db33da384f1dde135...,032d5a4b5a6a344ca15f6284e3e149f4716a1af782ffbb...,e960fd8385a1603003727,tka.jp,10000000.0,100000,0.0,0.000000


In [17]:
DF_channels[DF_channels["amount"]> DF_channels["capacity"]]

,source,destination,capacity,amount


In [29]:
DF_fixed = pd.DataFrame()
c = 0
for index, value in model.x.extract_values().items():
    if value != 0:
        DF_fixed.loc[c, "source"] = index[0]
        DF_fixed.loc[c, "destination"] = index[1]
        DF_fixed.loc[c, "used"] = Decimal(value)
        c += 1

DF_fixed_pos = DF_fixed[DF_fixed["used"]!=0]
DF_fixed_pos

,source,destination,used
0,0205198c099c45acedf988445f71da087ca39cd80847e5...,03d65c1c321cf8417e1565fb654896219b6fae33eeb62d...,1
1,02062a64c5d381c77f8ef679133e6207b995d33914c97c...,03bbdf5faaa9cb0ef87fca90fe31df13b9ca6203c26455...,1
2,0207fded7f81647771ac646e49e9a029c956e8de911fa6...,025d28dc4c4f5ce4194c31c3109129cd741fafc1ff2f6e...,1
3,020b831768ded7615eef25b34639c14b912496137ff521...,02eb34fafb08611c48cb3f220121e5278f29b1e122f677...,1
4,020fc7e62c9956a34bcdba309a4809ccfd7adcae4b8b64...,02fcc5bfc48e83f06c04483a2985e1c390cb0f35058baa...,1
...,...,...,...
227,03ecb39d80deaf6a934bac97ba514ef9709560e765c640...,0379221a4051d4171490e43e4a09e218a02941f06996b7...,1
228,03ed75f1ec977b78f26e509cda991f28d30fa6ee6138d9...,02c2fc4df9c9a3480a9ead094c19fd3a64888e09372827...,1
229,03fbe1c1baedbc99b2642ae524d9c2a6f12b771a3ab91e...,02df315ed19636d66272e2ebbc1f413e460ac5a004810b...,1
230,03fc8c900ee8f98e056ca2f2865466fed915dc61757b1a...,02830c79a405aebe208ffd005397689e02cd04f0438f77...,1


In [31]:
intersection = DF_fixed_pos.merge(DF_channels_pos, on=["source", "destination"], how="outer")
intersection

,source,destination,used,source-alias,destination-alias,capacity,amount,base_fee,rate_fee
0,0205198c099c45acedf988445f71da087ca39cd80847e5...,03d65c1c321cf8417e1565fb654896219b6fae33eeb62d...,1,NaN,NaN,NaN,NaN,NaN,NaN
1,02062a64c5d381c77f8ef679133e6207b995d33914c97c...,03bbdf5faaa9cb0ef87fca90fe31df13b9ca6203c26455...,1,NaN,NaN,NaN,NaN,NaN,NaN
2,0207fded7f81647771ac646e49e9a029c956e8de911fa6...,025d28dc4c4f5ce4194c31c3109129cd741fafc1ff2f6e...,1,NaN,NaN,NaN,NaN,NaN,NaN
3,020b831768ded7615eef25b34639c14b912496137ff521...,02eb34fafb08611c48cb3f220121e5278f29b1e122f677...,1,NaN,NaN,NaN,NaN,NaN,NaN
4,020fc7e62c9956a34bcdba309a4809ccfd7adcae4b8b64...,02fcc5bfc48e83f06c04483a2985e1c390cb0f35058baa...,1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
227,03ecb39d80deaf6a934bac97ba514ef9709560e765c640...,0379221a4051d4171490e43e4a09e218a02941f06996b7...,1,NaN,NaN,NaN,NaN,NaN,NaN
228,03ed75f1ec977b78f26e509cda991f28d30fa6ee6138d9...,02c2fc4df9c9a3480a9ead094c19fd3a64888e09372827...,1,NaN,NaN,NaN,NaN,NaN,NaN
229,03fbe1c1baedbc99b2642ae524d9c2a6f12b771a3ab91e...,02df315ed19636d66272e2ebbc1f413e460ac5a004810b...,1,NaN,NaN,NaN,NaN,NaN,NaN
230,03fc8c900ee8f98e056ca2f2865466fed915dc61757b1a...,02830c79a405aebe208ffd005397689e02cd04f0438f77...,1,NaN,NaN,NaN,NaN,NaN,NaN
